## 요약
- 기존 전처리 코드에서는 정수 인코딩한 token들도 scaling을 진행함
- transformer에서 embedding model을 사용하려면 정수 값으로 입력 받아야 함
- 또한 중고 도서 데이터가 아닌 도서 정보만 사용
- 이에 맞게 코드 수정

In [276]:
import os, natsort, re
from tqdm import tqdm
import time, random

In [277]:
from itertools import repeat, chain

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("fivethirtyeight")
# 이미지가 레이아웃 안으로 들어오도록 함
mpl.rcParams.update({"figure.constrained_layout.use": True})
mpl.rcParams['axes.unicode_minus'] = False

In [4]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [4]:
#cd /content/drive/MyDrive/AI3_prjct2_aladin/

In [3]:
cd /content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/

[Errno 2] No such file or directory: '/content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/'
/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_book_price/research


In [4]:
# 로컬에서

plt.rc("font", family = "D2Coding")
plt.rcParams["axes.unicode_minus"] = False

In [278]:
PRJCT_PATH = '/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_book_price/'
#PRJCT_PATH = '/content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/'
#PRJCT_PATH = '/content/drive/MyDrive/AI3_prjct2_aladin/aladin_usedbook/'
save_dir = 'processed/model_input'
dir_path = os.path.join(PRJCT_PATH,save_dir)
#dir_path = './'

In [279]:
ls

20250211_reg_simulation.ipynb:Zone.Identifier
240710_crawling_step0.ipynb
240711_crawling_step1.ipynb
240711_preprocess_bookinfo.ipynb
240715_encoding_usedinfo.ipynb
240716_check_bookinfo.ipynb
240716_check_bookinfo2.ipynb
240716_encoding_bookinfo.ipynb
240717_simple_model_for_sample.ipynb
240717_split_and_scale.ipynb
240718_step0_by_js.ipynb
240719_additional_eda.ipynb
240719_simple_model_for_cropped.ipynb
240721_GridSearch_for_XGB.ipynb
240721_experiment_w_XGB.ipynb
240721_hyperparameters_XGB.ipynb
241023_basic_model.ipynb
241023_preprocess_for_pred_rglr.ipynb
241024_basic_torch_model.ipynb
250112_modulized_test_20480_init_lr_2.12.ipynb
250112_modulized_test_20480_init_lr_2.12.ipynb:Zone.Identifier
250112_modulized_test_20480_init_lr_analyze2.ipynb
250112_modulized_test_20480_init_lr_analyze2.ipynb:Zone.Identifier
250112_modulized_test_plot_rslt.ipynb
250112_modulized_test_plot_rslt.ipynb:Zone.Identifier
250123_modulized_test_20480_init_lr_test_score.ipynb
250123_modulized_test_20480

In [280]:
import sys
sys.path.append(PRJCT_PATH)

In [281]:
RSLT_DIR = PRJCT_PATH + 'processed/'

bookinfo_name = 'bookinfo_ver{}.pkl'.format(0.75)
bookinfo_path = os.path.join(RSLT_DIR,bookinfo_name)

sys.path.append(PRJCT_PATH)
from module_aladin.file_io import load_pkl, save_pkl
from module_aladin.data_process import pd_datetime_2_datenum

from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.sequence import pad_sequences
import itertools
from sklearn.preprocessing import MinMaxScaler

In [282]:
def set_corpus_size(freq,size_feat,mode):
    # 입력받은 mode와 size_feat에 따라 size 크기 결정
    if mode == 'uniform':
        cond = freq['counts']>=freq['counts'].iloc[size_feat]
        size = np.sum(cond)
    elif mode =='ths':
        cond = freq[freq['counts'] > size_feat]
        size = np.sum(cond)
    else :
        if size_feat == None : size = len(freq)
#        elif size_feat > len(data) : size = len(freq)
        else : size = size_feat
    return size

def make_encoding_by_freq(freq,null_val='[PAD]',size_feat=None,mode=None):
    #빈도수 기반 정수 인코딩 dict 만들기
    # freq : token 별 등장 빈도 (value_count), size_feat : size관련 변수(max_size, ths등), mode : size 결정 방법
    df_freq = pd.DataFrame(freq).T
    df_freq = df_freq.rename(columns={0:'token',1:'counts'})
    temp = df_freq.sort_values(by='counts',ascending=False)
    size = set_corpus_size(temp,size_feat,mode)
    temp = temp.iloc[:size]
    temp['val'] = np.arange(size)+1
    temp2 = temp.set_index('token').to_dict()
    map_token_encode = temp2['val']
    map_token_encode[null_val]=0
    return map_token_encode

def encode_tokens(map_token,x,oov=True):
    oov_val = len(map_token)+1 if oov else 0 
    return map_token[x] if x in map_token else oov_val

def make_author_encode_map(bookinfo,ths_author):
    pvtb = pd.pivot_table(data=bookinfo,index='Author',values='SalesPoint',aggfunc=np.sum)
    pvtb = pvtb.sort_values(by='SalesPoint',ascending=False)
    author_top_k= pvtb[pvtb['SalesPoint']>=ths_author].index
    encode_author = pd.DataFrame({'author' : author_top_k.values,'val':np.arange(1,len(author_top_k)+1)})
    encode_author = encode_author.set_index('author')
    return encode_author.to_dict()['val']

def make_publshr_encode_map(publshr_data,ths_publshr):
    stats = publshr_data.value_counts().sort_values(ascending=False)
    top_k_val = stats.iloc[ths_publshr]
    publshr_top_k = list(stats[stats >= top_k_val].index)
    return {
        publshr : n+1
        for n,publshr in enumerate(publshr_top_k)
    }
    

def make_store_encode_map(store_data):
    stores= store_data.value_counts().sort_values(ascending=False)
    return {
        place : n+1
        for n,place in enumerate(stores.index)
    }

In [283]:
file_name = 'bookinfo_ver{}.csv'.format(1.0)
file_path = os.path.join(RSLT_DIR,file_name)
bookinfo_raw = pd.read_csv(file_path)

In [284]:
cols = bookinfo_raw.columns.to_list()
x_idxs, y_idx = [0,2,3,4,5,6,9,10], 7
x_cols = [cols[i] for i in x_idxs]
y_col = cols[y_idx]

data_X, data_y  = bookinfo_raw[x_cols], bookinfo_raw[y_col]

In [285]:
from sklearn.model_selection import train_test_split

X_data, X_tst, y_data, y_tst = train_test_split(data_X,data_y,test_size=0.2,random_state=329)
X_trn, X_vld, y_trn, y_vld = train_test_split(X_data,y_data,test_size=0.2,random_state=329)

display(X_trn.columns)
display(X_trn.shape, y_trn.shape)
display(X_vld.shape, y_vld.shape)
display(X_tst.shape, y_tst.shape)

Index(['BName', 'BName_sub', 'Author', 'Publshr', 'Author_mul', 'Pdate',
       'SalesPoint', 'Category'],
      dtype='object')

(101173, 8)

(101173,)

(25294, 8)

(25294,)

(31617, 8)

(31617,)

In [286]:
data_dict = {
    'trn': {
        'X': X_trn,
        'y': y_trn
        },
    'vld':{
        'X': X_vld,
        'y': y_vld
        },
    'tst':{
        'X': X_tst,
        'y': y_tst
        
    }
}

In [287]:
display(data_dict['trn']['X'].info())
display(data_dict['vld']['X'].info())
display(data_dict['tst']['X'].info())

<class 'pandas.core.frame.DataFrame'>
Index: 101173 entries, 96986 to 69788
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BName       101172 non-null  object
 1   BName_sub   5760 non-null    object
 2   Author      101173 non-null  object
 3   Publshr     101173 non-null  object
 4   Author_mul  101173 non-null  bool  
 5   Pdate       101173 non-null  int64 
 6   SalesPoint  101173 non-null  int64 
 7   Category    101173 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 6.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 25294 entries, 132734 to 43660
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BName       25294 non-null  object
 1   BName_sub   1434 non-null   object
 2   Author      25294 non-null  object
 3   Publshr     25294 non-null  object
 4   Author_mul  25294 non-null  bool  
 5   Pdate       25294 non-null  int64 
 6   SalesPoint  25294 non-null  int64 
 7   Category    25294 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 1.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 31617 entries, 146028 to 115151
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BName       31617 non-null  object
 1   BName_sub   1813 non-null   object
 2   Author      31617 non-null  object
 3   Publshr     31617 non-null  object
 4   Author_mul  31617 non-null  bool  
 5   Pdate       31617 non-null  int64 
 6   SalesPoint  31617 non-null  int64 
 7   Category    31617 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 2.0+ MB


None

In [288]:
mecab = Mecab()
tokenizer_basic = lambda x : mecab.morphs(x)
cut_date = lambda x : [x[:4],x[4:6],x[6:]]
#apply tokenizer
cols_tknz = ['Category','BName','BName_sub']
cols_freq = ['Author','Publshr']

In [289]:
for mode, data in data_dict.items():
    bookinfo = data['X']
    for col in cols_tknz:
        bookinfo[col] = bookinfo[col].fillna('').apply(tokenizer_basic)
    bookinfo['Pdate'] = bookinfo['Pdate'].astype(str).apply(cut_date)
    bookinfo['Author_mul'] = bookinfo['Author_mul'].astype(int).astype(str)
    data_dict[mode]['X'] = bookinfo

In [290]:
#아래 ths 는 EDA 결과 제가 자의적으로 정한 내용
bookinfo = data_dict['trn']['X']
ths_author = int(np.round(len(bookinfo)/500)*75)
ths_publshr = int(np.round(len(bookinfo)/500)*5)

map_author_encode = make_author_encode_map(bookinfo[['Author','SalesPoint']],ths_author)
map_publshr_encode = make_publshr_encode_map(bookinfo['Publshr'],ths_publshr)

encode_maps = {
    'Author' : lambda x : encode_tokens(map_author_encode,x,oov=False),
    'Publshr' : lambda x : encode_tokens(map_publshr_encode,x,oov=False),
}

In [291]:
for col in tqdm(cols_freq):
    bookinfo[col] = bookinfo[col].map(encode_maps[col]).astype(str)

100%|██████████| 2/2 [00:00<00:00, 14.18it/s]


In [292]:
#make encoding map
cols_vec = cols_tknz + ['Pdate']
cols_tknz2= ['BName', 'BName_sub', 'Author', 'Publshr', 'Author_mul', 'Pdate','Category']
book_tknzed = bookinfo[cols_tknz2].to_dict('series')
#book_name, book_subname, category = book_tknzed['BName'], book_tknzed['BName_sub'],book_tknzed['Category']

In [293]:
display('268' in book_tknzed['Publshr'].values)
display(book_tknzed['Publshr'].values)

True

array(['19', '92', '37', ..., '392', '0', '0'], dtype=object)

In [294]:
cols_tknz2

['BName', 'BName_sub', 'Author', 'Publshr', 'Author_mul', 'Pdate', 'Category']

In [295]:
tkn_vals=[]
for col in cols_tknz2:
    if col in cols_vec:
        tkn_vals.append(np.array(list(itertools.chain(*book_tknzed[col].values))))
    else : tkn_vals.append(np.array(list(itertools.chain(book_tknzed[col].values))))
#tokens = np.concatenate(tkn_vals,axis=0)
#tokens
tkn_vals

[array(['니세코', '이', '비하인드', ..., '원리', '와', '이론'], dtype='<U15'),
 array(['(', '전', '작품', ..., '특별', '판', ')'], dtype='<U13'),
 array(['0', '0', '0', ..., '0', '0', '0'], dtype='<U4'),
 array(['19', '92', '37', ..., '392', '0', '0'], dtype='<U4'),
 array(['1', '0', '0', ..., '1', '0', '0'], dtype='<U1'),
 array(['2015', '05', '07', ..., '2011', '03', '21'], dtype='<U4'),
 array(['소설', '/', '시', ..., '/', '전문', '서적'], dtype='<U3')]

In [296]:
tokens = np.concatenate(tkn_vals,axis=0)

In [297]:
tokens_ds = pd.Series(tokens)
cond_numeric = ~(pd.to_numeric(tokens_ds,errors='coerce')).isna()

tokens_num = tokens[cond_numeric]
tokens_str = tokens[~cond_numeric]


In [298]:
print(len(tokens_num),len(tokens_str))
print(len(np.unique(tokens_num)),len(np.unique(tokens_str)))

657355 822353
1769 31441


In [299]:
temp,cnt = np.unique(tokens_num,return_counts=True)
temp2 = np.apply_along_axis(lambda x : [(len(n),cnt[i]) for i,n in enumerate(x)],axis=0,arr=temp)
temp2

array([[     1, 157443],
       [     2,      9],
       [     4,      1],
       ...,
       [     4,      2],
       [     3,     69],
       [     4,      3]])

In [300]:
np.sum(temp2[:,1][temp2[:,0] < 3])/np.sum(cnt)

0.7452731020529242

In [305]:
tkns_num = 31000
token_freq_str = np.unique(tokens_str,return_counts=True)
token_freq_num = np.unique(tokens_num,return_counts=True)
map_token_encode_str = make_encoding_by_freq(token_freq_str,size_feat=tkns_num)
map_token_encode_num = make_encoding_by_freq(token_freq_num)

In [306]:
for key,val in map_token_encode_num.items():
    if key == '[PAD]' : continue
    map_token_encode_num[key] = val+tkns_num

map_token_encode_total = dict(map_token_encode_str,**map_token_encode_num)

In [307]:
encode_1line =lambda x: list(map(lambda y : encode_tokens(map_token_encode_total,y),x))

In [308]:
encode_1line(['268','2341252345','askdlfjwef'])

[31375, 32771, 32771]

In [309]:
maxlens={
    'Category' : 5,
    'BName' : 28,
    'BName_sub' : 25,
    'Pdate':3
}
x_cols = ['BName', 'BName_sub', 'Author',
   'Author_mul', 'Publshr', 'Pdate','Category']

book_cols = ['BName', 'BName_sub', 'Author', 'Author_mul', 'Publshr', 'Pdate',
       'RglPrice', 'SalesPoint', 'Category']
xcols_scalar = list(filter(lambda x : x not in maxlens.keys(),x_cols)) 

In [310]:
#encode X
X_encoded=dict()
for mode,sample in tqdm(data_dict.items()):
    X_mode = sample['X']
    #padding and encoding
    encoded = pd.DataFrame(index=X_mode.index) 
    for col in x_cols:
        if col in maxlens:
            padded = pad_sequences(X_mode[col],padding='post',
                                   maxlen=maxlens[col],
                                   value='[PAD]',dtype=object)
            encoded[col] = list(np.apply_along_axis(encode_1line,0,padded))
        else :
            encoded[col] = encode_1line(X_mode[col].values)

    concat_tknzed =np.apply_along_axis(np.hstack,1,encoded[list(maxlens.keys())].values)
    x_scalar = encoded[xcols_scalar].values
    X = np.hstack((concat_tknzed,x_scalar))
    X_encoded[mode] = X    

100%|██████████| 3/3 [00:08<00:00,  2.87s/it]


In [311]:
data_dict['trn']['X'].iloc[74310]

BName         [눈, 이, 아닌, 것, 으로, 도, 읽, 은, 기분]
BName_sub                                 []
Author                                     0
Publshr                                  268
Author_mul                                 0
Pdate                         [2017, 12, 30]
SalesPoint                               666
Category                               [에세이]
Name: 114793, dtype: object

In [312]:
data_dict['trn']['X'].iloc[74310,3], '268' in data_dict['trn']['X']['Publshr'].values

('268', True)

In [313]:
map_token_encode_num['268']

31375

In [314]:
display(np.sum(np.isnan(X_encoded['trn'].astype(np.float64))))
display(np.sum(np.isnan(X_encoded['vld'].astype(np.float64))))
display(np.sum(np.isnan(X_encoded['tst'].astype(np.float64))))

0

0

0

In [315]:
print(len(np.unique(X_encoded['trn'])))
print(np.max(X_encoded['trn']))
print(np.min(X_encoded['trn']))

32759
32771
0


In [242]:
X_coded = {
    mode : data.astype(np.int32)
    for mode, data in X_encoded.items()
} 

In [245]:
data_type = 'whole-encoded'
strat=0
ver=1.0
dir_path = os.path.join(RSLT_DIR,'model_input')
for mode, x_scaled in X_coded.items():
    save_pkl(dir_path,'{}.v{}_st-{}_X_{}.pkl'.format(data_type,ver,strat,mode),x_scaled)
for mode,sample in data_dict.items(): 
    save_pkl(dir_path,'{}.v{}_st-{}_y_{}.pkl'.format(data_type,ver,strat,mode),sample['y'].to_numpy())